# Table of Contents
 <p><div class="lev1"><a href="#Optimize-Dynamic-Subgraphs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Optimize Dynamic Subgraphs</a></div><div class="lev2"><a href="#Initialize-Environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2"><a href="#Generate-List-of-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev2"><a href="#Construct-Configuration-Matrices-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Construct Configuration Matrices</a></div><div class="lev2"><a href="#Parameter-Search-Space-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Parameter Search Space</a></div><div class="lev2"><a href="#Run-Non-Negative-Matrix-Factorization-Algorithm-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Run Non-Negative Matrix Factorization Algorithm</a></div><div class="lev2"><a href="#Analyze-Parameter-Search-Space-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Analyze Parameter Search Space</a></div><div class="lev3"><a href="#Display-Parameter-Search-Space-1.6.1"><span class="toc-item-num">1.6.1&nbsp;&nbsp;</span>Display Parameter Search Space</a></div><div class="lev3"><a href="#Display-Relationship-Between-Parameters-and-Quality-Measures-1.6.2"><span class="toc-item-num">1.6.2&nbsp;&nbsp;</span>Display Relationship Between Parameters and Quality Measures</a></div><div class="lev3"><a href="#Display-Bivariate-Distribution-of-Error-and-Sparsity-1.6.3"><span class="toc-item-num">1.6.3&nbsp;&nbsp;</span>Display Bivariate Distribution of Error and Sparsity</a></div><div class="lev3"><a href="#Find-Optimal-Parameters-1.6.4"><span class="toc-item-num">1.6.4&nbsp;&nbsp;</span>Find Optimal Parameters</a></div><div class="lev3"><a href="#Plot-closest-optimization-example-1.6.5"><span class="toc-item-num">1.6.5&nbsp;&nbsp;</span>Plot closest optimization example</a></div><div class="lev1"><a href="#Detect-Dynamic-Subgraphs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Detect Dynamic Subgraphs</a></div><div class="lev2"><a href="#Initialize-Environment-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2"><a href="#Run-Non-Negative-Matrix-Factorization-Algorithm-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Run Non-Negative Matrix Factorization Algorithm</a></div><div class="lev2"><a href="#Consensus-NMF-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Consensus NMF</a></div><div class="lev2"><a href="#Consensus-Subgraphs-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Consensus Subgraphs</a></div><div class="lev2"><a href="#Subgraphs-of-Brain-Systems-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Subgraphs of Brain Systems</a></div><div class="lev3"><a href="#Formulate-ROI-Dict-2.5.1"><span class="toc-item-num">2.5.1&nbsp;&nbsp;</span>Formulate ROI Dict</a></div><div class="lev3"><a href="#Convert-ROI-Subgraphs-to-Brain-System-Subgraphs-2.5.2"><span class="toc-item-num">2.5.2&nbsp;&nbsp;</span>Convert ROI Subgraphs to Brain System Subgraphs</a></div><div class="lev3"><a href="#Plot-all-Brain-System-Subgraphs-2.5.3"><span class="toc-item-num">2.5.3&nbsp;&nbsp;</span>Plot all Brain System Subgraphs</a></div>

# Optimize Dynamic Subgraphs

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    
except:
    print 'NOT IPYTHON'

from __future__ import division
from IPython.display import display

import os
import sys
import glob
import json
import subprocess

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

import fig_plotting
rcParams = fig_plotting.update_rcparams(rcParams)

import scipy.stats as stats

os.chdir('../')
import Codebase
conv_adj_matr_to_cfg_matr = Codebase.Networks.configuration.convert_adj_matr_to_cfg_matr
conv_cfg_vec_to_adj_matr = Codebase.Networks.configuration.convert_conn_vec_to_adj_matr
os.chdir('./Analysis Notebooks/')

path_CoreData = '/home/akhambhati/JagHome/hoth_research/CoreData/Sync_Cog_Control-Medaglia'
path_PeriphData = '/home/akhambhati/JagHome/hoth_research/PeriphData/ds-NMF_CogControl'
path_InpData = path_PeriphData + '/e01-FuncNetw'
path_ExpData = path_PeriphData + '/e02-FuncSubg'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

## Generate List of Data

In [ ]:
inp_fname = glob.glob('{}/*.npz'.format(path_InpData))

expr_dict = {}
for fname in inp_fname:
    subj_id = fname.split('/')[-1].split('.')[0]
    expr_id = fname.split('/')[-1].split('.')[1]
    
    try:
        expr_dict[expr_id]['adj_files'].append(fname)
    except:
        expr_dict[expr_id] = {}
        expr_dict[expr_id]['adj_files'] = []
        expr_dict[expr_id]['adj_files'].append(fname)

## Construct Configuration Matrices

In [ ]:
key_ord = ['adj_rs_pos', 'adj_rs_neg',
           'adj_lo_pos', 'adj_lo_neg',
           'adj_hi_pos', 'adj_hi_neg']

cfg_list = []
cfg_key = []
for expr_id in expr_dict.keys():    
    for fname in expr_dict[expr_id]['adj_files']:
        df = np.load(fname)
        
        for key in key_ord:
            cfg_matr = conv_adj_matr_to_cfg_matr(df[key])
            for cfg_vec in cfg_matr:
                cfg_list.append(cfg_vec)
                cfg_key.append('{}_{}'.format(key, expr_id))
        
np.savez('{}/NMF_Optimization.CfgMatr.npz'.format(path_ExpData),
         cfg_matr=np.array(cfg_list),
         cfg_key=np.array(cfg_key))

## Parameter Search Space

In [ ]:
# Set search ranges
search_alpha_log = [0.01, 2.0]
search_beta_log = [0.01, 2.0]
search_rank = [2, 61]
n_search = 10000

# Generate parameter set
param_set = []
param_alpha = np.zeros(n_search)
param_beta = np.zeros(n_search)
param_rank = np.zeros(n_search)
for ii in xrange(n_search):
    alpha = np.random.uniform(low=search_alpha_log[0],
                              high=search_alpha_log[1])

    beta = np.random.uniform(low=search_beta_log[0],
                             high=search_beta_log[1])
    
    rank = np.random.randint(low=search_rank[0],
                             high=search_rank[1])
    
    param_alpha[ii] = alpha
    param_beta[ii] = beta
    param_rank[ii] = rank
    param_set.append({'id': ii+1,
                      'alpha': alpha,
                      'beta': beta,
                      'rank': rank})

    
# Display Joint-Distributions Parameter Space
%matplotlib inline
g = sns.jointplot(param_alpha, param_beta, kind='kde')
g.set_axis_labels('Alpha', 'Beta')

g = sns.jointplot(param_alpha, param_rank, kind='kde')
g.set_axis_labels('Alpha', 'Rank')

g = sns.jointplot(param_beta, param_rank, kind='kde')
g.set_axis_labels('Beta', 'Rank')

np.savez('{}/NMF_Optimization.Params.npz'.format(path_ExpData), param_set=param_set)

## Run Non-Negative Matrix Factorization Algorithm
*__WARNING: Will Delete Existing Output__*

In [ ]:
# Remove all existing output (retains pipe/pipeline definitions)
rm_outp = glob.glob("{}/NMF_Optimization.Param_*".format(path_ExpData))

for rm_type in [rm_outp]:
    for path in rm_type:
        try:
            os.remove(path)
        except:
            print("{} not found".format(path))

In [ ]:
param_set = np.load('{}/NMF_Optimization.Params.npz'.format(path_ExpData))['param_set']
path_cfg_matr = glob.glob('{}/NMF_Optimization.CfgMatr.npz'.format(path_ExpData))[0]

proc_item = []
for param in param_set:
    res_path = '{}/NMF_Optimization.Param_{}.npz'.format(path_ExpData, param['id'])
    if os.path.exists(res_path):
        continue
    else:
        proc_item.append({'path_cfg_matr': path_cfg_matr,
                          'param': param})
print('There are {} jobs to process.'.format(len(proc_item)))
    
# Submit proc_item list as jobs to qsub
job_file = open('./e02-job-NMF_Optimization', 'w')
job_file.write('#!/bin/bash\n\n')
for pitem in proc_item:    
    stdout_path = '{}/NMF_Optimization.Param_{}.stdout'.format(path_ExpData, pitem['param']['id'])
    stderr_path = '{}/NMF_Optimization.Param_{}.stderr'.format(path_ExpData, pitem['param']['id'])
    
    py_str = './pywrap-nmf_optimization {} {} {} {} {} {}'.format(
        pitem['path_cfg_matr'], pitem['param']['id'], 
        pitem['param']['alpha'], pitem['param']['beta'],
        pitem['param']['rank'], path_ExpData)

    cmd_str = 'qsub -cwd -o {} -e {} -l h_vmem=6.1G,s_vmem=6G'.format(stdout_path, stderr_path)

    job_file.write('{} {}\n'.format(cmd_str, py_str))
job_file.close()    

## Analyze Parameter Search Space

In [ ]:
import scipy.stats as stats

param_set = np.load('{}/NMF_Optimization.Params.npz'.format(path_ExpData))['param_set']
param_run = glob.glob('{}/NMF_Optimization.Param_*.npz'.format(path_ExpData))
n_param = len(param_run)

param_alpha = np.zeros(n_param)
param_beta = np.zeros(n_param)
param_rank = np.zeros(n_param)

err = np.zeros(n_param)
subnet_sparsity = np.zeros(n_param)
coef_sparsity = np.zeros(n_param)

ii = 0
for path in param_run:
    if (ii % 500) == 0:
        print('Processed optimization: {} of {}'.format(ii, n_param))
    
    try:
        data = np.load(path, mmap_mode='r')
    except:
        continue
    
    p_id = int(path.split('/')[-1].split('.')[1].split('_')[1])
    param_alpha[ii] = param_set[p_id-1]['alpha']
    param_beta[ii] = param_set[p_id-1]['beta']
    param_rank[ii] = param_set[p_id-1]['rank']

    err[ii] = data['err'][-1]
    subnet_sparsity[ii] = (data['fac_subnet'] == 0).mean()
    coef_sparsity[ii] = (data['fac_coef'] == 0).mean()
    
    ii += 1
    data.close()

param_alpha = param_alpha[:ii]
param_beta = param_beta[:ii]
param_rank = param_rank[:ii]

err = err[:ii]
subnet_sparsity = subnet_sparsity[:ii]
coef_sparsity = coef_sparsity[:ii]

### Display Parameter Search Space

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(param_alpha, param_beta, param_rank, alpha=0.3)
ax.set_xlabel('Alpha')
ax.set_ylabel('Beta')
ax.set_zlabel('Rank')

### Display Relationship Between Parameters and Quality Measures

In [ ]:
%matplotlib inline

for qmeas, qname in [(err, 'Residual Error'),
                     (subnet_sparsity, 'Pct Subgraph Sparse'),
                     (coef_sparsity, 'Pct Coef Sparse')]:
    print('Plotting -- {}'.format(qname))
    
    g = sns.jointplot(param_rank, qmeas, kind='kde',
                       ylim=[np.min(qmeas), np.max(qmeas)])
    g.set_axis_labels('Rank', qname)

    g = sns.jointplot((param_alpha), qmeas, kind='kde',
                       ylim=[np.min(qmeas), np.max(qmeas)])
    g.set_axis_labels('Alpha', qname)

    g = sns.jointplot((param_beta), qmeas, kind='kde',
                       ylim=[np.min(qmeas), np.max(qmeas)])
    g.set_axis_labels('Beta', qname)

### Display Bivariate Distribution of Error and Sparsity

In [ ]:
g = sns.jointplot(subnet_sparsity, err, kind='kde',
                  xlim=[np.min(subnet_sparsity), np.max(subnet_sparsity)],
                  ylim=[np.min(err), np.max(err)])
g.set_axis_labels('Pct Subgraph Sparse', 'Residual Error')

g = sns.jointplot(coef_sparsity, err, kind='kde',
                  xlim=[np.min(coef_sparsity), np.max(coef_sparsity)],
                  ylim=[np.min(err), np.max(err)])
g.set_axis_labels('Pct Coef Sparse', 'Residual Error')

g = sns.jointplot(subnet_sparsity, coef_sparsity, kind='kde',
                  xlim=[np.min(subnet_sparsity), np.max(subnet_sparsity)],
                  ylim=[np.min(coef_sparsity), np.max(coef_sparsity)])
g.set_axis_labels('Pct Subgraph Sparse', 'Pct Coef Sparse')

### Find Optimal Parameters

Minimize the residual error and maximize subgraph and temporal coefficient sparsity

In [ ]:
# Specify ranges
err_range = [np.percentile(err, 0), np.percentile(err, 25)]
subnet_sparsity_range = [np.percentile(subnet_sparsity, 75), np.percentile(subnet_sparsity, 100)]
coef_sparsity_range = [np.percentile(coef_sparsity, 75), np.percentile(coef_sparsity, 100)]

err_ix = np.flatnonzero((err >= err_range[0]) & (err <= err_range[1]))
subnet_sparsity_ix = np.flatnonzero((subnet_sparsity >= subnet_sparsity_range[0]) &
                                    (subnet_sparsity <= subnet_sparsity_range[1]))
coef_sparsity_ix = np.flatnonzero((coef_sparsity >= coef_sparsity_range[0]) &
                                    (coef_sparsity <= coef_sparsity_range[1]))

param_ix = np.intersect1d(np.intersect1d(err_ix, subnet_sparsity_ix), coef_sparsity_ix)
n_param = len(param_ix)

print('Alpha: {} +/- {}'.format(np.mean(param_alpha[param_ix]),
                                np.std(param_alpha[param_ix]) / np.sqrt(n_param)))
print('Beta: {} +/- {}'.format(np.mean(param_beta[param_ix]),
                               np.std(param_beta[param_ix]) / np.sqrt(n_param)))
print('Rank: {} +/- {}'.format(np.mean(param_rank[param_ix]),
                               np.std(param_rank[param_ix]) / np.sqrt(n_param)))

### Plot closest optimization example

In [ ]:
# Find param id that is a close match to optimal
opt_alpha = 0.95
opt_beta = 0.90
opt_rank = 54

param_set = np.load('{}/NMF_Optimization.Params.npz'.format(path_ExpData))['param_set']
param_rank = np.array([p['rank'] for p in param_set])
param_beta = np.array([p['beta'] for p in param_set])
param_alpha = np.array([p['alpha'] for p in param_set])

opt_rank_ix = np.flatnonzero(param_rank == opt_rank)
opt_beta_ix = np.argsort(np.abs(param_beta - opt_beta))[:2000]
opt_alpha_ix = np.argsort(np.abs(param_alpha - opt_alpha))[:2000]


opt_ix = np.intersect1d(np.intersect1d(opt_alpha_ix, opt_beta_ix), opt_rank_ix)[-1]
print(param_rank[opt_ix],
      param_alpha[opt_ix],
      param_beta[opt_ix])

print('Plotting All Subgraphs...')
df = np.load('{}/NMF_Optimization.Param_{}.npz'.format(path_ExpData, opt_ix+1))
rank = df['fac_subnet'].shape[0]
n_col = int(np.sqrt(rank))
n_row = int(np.ceil(rank / n_col))

%matplotlib inline
plt.figure(figsize=(16, 12))
for ii in xrange(opt_rank):
    ax = plt.subplot(n_row, n_col, ii+1)
    ax.matshow(conv_cfg_vec_to_adj_matr(df['fac_subnet'][ii, :]), cmap='rainbow')
    ax.set_axis_off()
plt.show()

# Detect Dynamic Subgraphs

## Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    
except:
    print 'NOT IPYTHON'

from __future__ import division
from IPython.display import display

import os
import sys
import glob
import json
import subprocess

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

import fig_plotting
rcParams = fig_plotting.update_rcparams(rcParams)

import scipy.stats as stats

os.chdir('../')
import Codebase
conv_adj_matr_to_cfg_matr = Codebase.Networks.configuration.convert_adj_matr_to_cfg_matr
conv_cfg_vec_to_adj_matr = Codebase.Networks.configuration.convert_conn_vec_to_adj_matr
os.chdir('./Analysis Notebooks/')

path_CoreData = '/home/akhambhati/JagHome/hoth_research/CoreData/Sync_Cog_Control-Medaglia'
path_PeriphData = '/home/akhambhati/JagHome/hoth_research/PeriphData/ds-NMF_CogControl'
path_InpData = path_PeriphData + '/e01-FuncNetw'
path_ExpData = path_PeriphData + '/e02-FuncSubg'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

## Run Non-Negative Matrix Factorization Algorithm
*__WARNING: Will Delete Existing Output__*

In [ ]:
# Remove all existing output (retains pipe/pipeline definitions)
rm_outp = glob.glob("{}/NMF_Optimized.Seed_*".format(path_ExpData))

for rm_type in [rm_outp]:
    for path in rm_type:
        try:
            os.remove(path)
        except:
            print("{} not found".format(path))

In [ ]:
# Optimal Parameter Set
opt_alpha = 0.95
opt_beta = 0.90
opt_rank = 54
n_seed = 1000

param_set = []
for seed in xrange(n_seed):
    param_set.append({'id': seed+1,
                      'alpha': opt_alpha,
                      'beta': opt_beta,
                      'rank': opt_rank})
    

path_cfg_matr = glob.glob('{}/NMF_Optimization.CfgMatr.npz'.format(path_ExpData))[0]

proc_item = []
for param in param_set:
    res_path = '{}/NMF_Optimized.Seed_{}.npz'.format(path_ExpData, param['id'])
    if os.path.exists(res_path):
        continue
    else:
        proc_item.append({'path_cfg_matr': path_cfg_matr,
                          'param': param})
print('There are {} jobs to process.'.format(len(proc_item)))

# Submit proc_item list as jobs to qsub
job_file = open('./e02-job-NMF_Optimized', 'w')
job_file.write('#!/bin/bash\n\n')
for pitem in proc_item:    
    stdout_path = '{}/NMF_Optimized.Seed_{}.stdout'.format(path_ExpData, pitem['param']['id'])
    stderr_path = '{}/NMF_Optimized.Seed_{}.stderr'.format(path_ExpData, pitem['param']['id'])
    
    py_str = './pywrap-nmf_optimized {} {} {} {} {} {}'.format(
        pitem['path_cfg_matr'],
        pitem['param']['id'], pitem['param']['alpha'], pitem['param']['beta'],
        pitem['param']['rank'], path_ExpData)

    cmd_str = 'qsub -cwd -o {} -e {} -l h_vmem=6.1G,s_vmem=6G'.format(stdout_path, stderr_path)
    
    job_file.write('{} {}\n'.format(cmd_str, py_str))
job_file.close()    

## Consensus NMF

In [ ]:
from Codebase.Networks.SubgraphDetection.nonnegfac import nmf

path_cfg_matr = glob.glob('{}/NMF_Optimization.CfgMatr.npz'.format(path_ExpData))
seed_path = glob.glob('{}/NMF_Optimized.Seed_*.npz'.format(path_ExpData))
n_seed_block = 100
seed_path_blocks = np.random.permutation(seed_path).reshape(-1, n_seed_block)

for block_ix, seed_path_block in enumerate(seed_path_blocks):
    print('Processing Block: {}'.format(block_ix+1))

    # Concatenate subgraphs from block seeds
    fac_subnet_seeds = []
    for path_block in seed_path_block:
        data = np.load(path_block, mmap_mode='r')
        fac_subnet = data['fac_subnet'][:, :]
        data.close()

        n_fac = fac_subnet.shape[0]
        n_conn = fac_subnet.shape[1]

        for fac_ix in xrange(n_fac):
            fac_subnet_seeds.append(fac_subnet[fac_ix, :])
    fac_subnet_seeds = np.array(fac_subnet_seeds)

    n_obs = fac_subnet_seeds.shape[0]
    n_conn = fac_subnet_seeds.shape[1]
    
    if block_ix == 0:
        fac_subnet_init = np.random.uniform(low=0.0, high=1.0, size=(n_fac, n_conn))
    else:
        fac_subnet_init = fac_cons_subnet
    fac_coef_init = np.random.uniform(low=0.0, high=1.0, size=(n_fac, n_obs))

    # Consensus Subgraphs
    fac_cons_subnet, fac_cons_seeds, err = nmf.snmf_bcd(
        fac_subnet_seeds,
        alpha=0.0,
        beta=0.0,
        fac_subnet_init=fac_subnet_init,
        fac_coef_init=fac_coef_init,
        max_iter=100, verbose=True)

# Consensus Coefficients
data_cfg = np.load(path_cfg_matr, mmap_mode='r')
cfg_matr = np.nan_to_num(data_cfg['cfg_matr'][:, :])
n_win = cfg_matr.shape[0]
fac_cons_subnet_2, fac_cons_coef_2, err = nmf.snmf_bcd(
    cfg_matr,
    alpha=0.0,
    beta=0.0,
    fac_subnet_init=fac_cons_subnet,
    fac_coef_init=np.random.uniform(low=0.0, high=1.0, size=(n_fac, n_win)),
    max_iter=100, verbose=True)

# Cache the Consensus NMF result
np.savez("{}/NMF_Consensus.npz".format(path_ExpData),
         fac_subnet=fac_cons_subnet_2, fac_coef=fac_cons_coef_2, err=err)

## Consensus Subgraphs

In [ ]:
print('Plotting All Subgraphs...')
df = np.load('{}/NMF_Consensus.npz'.format(path_ExpData))
rank = df['fac_subnet'].shape[0]
n_col = int(np.sqrt(rank))
n_row = int(np.ceil(rank / n_col))

%matplotlib inline
plt.figure(figsize=(16, 12))
for ii in xrange(rank):
    ax = plt.subplot(n_row, n_col, ii+1)
    ax.matshow(conv_cfg_vec_to_adj_matr(df['fac_subnet'][ii, :]), cmap='rainbow')
    ax.set_axis_off()
plt.show()

## Subgraphs of Brain Systems

### Formulate ROI Dict

In [ ]:
df_parcel = pd.read_csv('{}/LausanneScale125.csv'.format(path_CoreData))

all_roi_names = []

roi_dict = {}
for lbl_id, lbl_roi, lbl_hemi in zip(df_parcel.Label_ID, df_parcel.ROI, df_parcel.Hemisphere):
    roi_name = '{}_{}'.format(lbl_hemi, lbl_roi.split('_')[0])
    #roi_name = lbl_roi.split('_')[0]
    
    if roi_name not in all_roi_names:
        all_roi_names.append(roi_name)

    try:
        roi_dict[roi_name]
    except KeyError:
        roi_dict[roi_name] = []

    roi_dict[roi_name].append(lbl_id-1)

### Convert ROI Subgraphs to Brain System Subgraphs

In [ ]:
bs_fac_subnet = np.zeros((rank, len(all_roi_names), len(all_roi_names)))

for fac_i, subg in enumerate(df['fac_subnet']):
    adj = conv_cfg_vec_to_adj_matr(subg)
    
    for ij_ii, ij_roi in enumerate(all_roi_names):
        for ik_ii, ik_roi in enumerate(all_roi_names):
            
            mean_conn = adj[roi_dict[ij_roi], :][:, roi_dict[ik_roi]].mean()
            bs_fac_subnet[fac_i, ij_ii, ik_ii] = mean_conn

### Plot all Brain System Subgraphs

In [ ]:
print('Plotting All Subgraphs...')
rank = bs_fac_subnet.shape[0]
n_col = int(np.sqrt(rank))
n_row = int(np.ceil(rank / n_col))

%matplotlib inline
plt.figure(figsize=(16, 12))
for ii in xrange(rank):
    ax = plt.subplot(n_row, n_col, ii+1)
    ax.matshow(bs_fac_subnet[ii, :, :], cmap='rainbow')
    ax.set_axis_off()
plt.show()

In [ ]:
plt.figure(figsize=(24,18))
sns.heatmap(bs_fac_subnet[15,:,:], cmap='rainbow', xticklabels=all_roi_names, yticklabels=all_roi_names)

In [ ]:
np.array(all_roi_names)[np.argsort(np.sum(bs_fac_subnet[15,:,:], axis=0))[::-1]]